In [5]:
#%% load modules
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
#%%
 # termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((9*6,3), np.float32) #6*7,3
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
#objp=objp*25 mm #scale by the square size 
 # Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images = glob.glob(r'C:\Users\amirs\OneDrive\Skrivebord\Mas513\Realsense_L515\Realsense_L515\*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2=cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),criteria)
        imgpoints.append(corners)
        # Draw and display the corners
#       cv2.drawChessboardCorners(img, (9,6), corners2,ret)
#       cv2.imshow('img',img)
#       cv2.waitKey(500)
        # Naming a window
        cv2.namedWindow("Resized_Window", cv2.WINDOW_NORMAL)
        # Using resizeWindow()
        cv2.resizeWindow("Resized_Window", 1800, 1080)
        # Displaying the image
        img_draw=cv2.drawChessboardCorners(img, (9,6), corners2,ret)
        cv2.imshow("Resized_Window", img_draw)
        cv2.waitKey(200)

cv2.destroyAllWindows()

objpoints=np.array(objpoints)
imgpoints=np.array(imgpoints)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
#mtx is intrinsic matrix; 
# rvecs is rotation matrix, 
# tvecs is translation matrix

np.savez(r'C.npz', mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs)
ret=np.array(ret)
mtx=np.array(mtx)
rvecs=np.array(rvecs)
tvecs=np.array(tvecs)
dist=np.array(dist)

print("Intrinsic Matrix (mtx):")
print(mtx)


#%% extrinsic calibration



def draw(img, corners, imgpts):
    corner = (tuple(corners[0].ravel()))
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

# Load previously saved data
with np.load(r'C.npz') as X:
    mtx, dist, _, _ = [X[i] for i in ('mtx','dist','rvecs','tvecs')]

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((9*6,3), np.float32) #6*7,3
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
#objp=objp*10
axis = np.float32([[10,0,0], [0,10,0], [0,0,-10]]).reshape(-1,3)
 # Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images = glob.glob(r'C:\Users\amirs\OneDrive\Skrivebord\Mas513\Realsense_L515\Realsense_L515\*.jpg')


for fname in images:
    #cv2.namedWindow("output", cv2.WINDOW_NORMAL)
    img = cv2.imread(fname)
    #imS = cv2.resize(img, (960, 540)) 
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    if ret == True:
        
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        objpoints=np.array(objpoints)
        imgpoints=np.array(imgpoints)

        # Find the rotation and translation vectors.
        ret, rvec, tvec = cv2.solvePnP(objp, corners2, mtx, dist)     #solvePnPRansac   , inlier
        #The solvePnP returns the object's origin in the camera's coordinate system given the  points  and the camera matrix.
        #rvec and tvec are the object's pose in the camera's coordinate system/frame 
        # transforming points from object frame to camera frame
        #transforms geometry from the object frame (input, right side) into the camera frame (output, left side)
        #expresses the object's pose relative to the camera frame i.e. gives T_obj^cam

        # Print translation vector (tvec) in x, y, z direction
        print("Translation vector (tvec) in x, y, z direction for this image:")
        print(f"x: {tvec[0][0]:.4f}, y: {tvec[1][0]:.4f}, z: {tvec[2][0]:.4f}")




        # Convert rotation vector (rvec) to rotation matrix
        rotation_matrix, _ = cv2.Rodrigues(rvec)
        # Get rotation angles (in degrees) from the rotation matrix
        rotation_angles = cv2.RQDecomp3x3(rotation_matrix)[0]  # rotation_angles is a tuple with (x, y, z)
        # Print rotation angles in x, y, z direction
        print("Rotation angles (degrees) in x, y, z direction for this image:")
        print(f"x: {rotation_angles[0]:.4f}, y: {rotation_angles[1]:.4f}, z: {rotation_angles[2]:.4f}")


        # Create the extrinsic matrix by horizontally appending the rotation matrix and translation vector
        extrinsic_matrix = np.hstack((rotation_matrix, tvec))

        # Print the extrinsic matrix (3x4)
        print("Extrinsic Matrix (3x4) for this image:")
        print(extrinsic_matrix)



        imgpts, jac = cv2.projectPoints(axis, rvec, tvec, mtx, dist) #objp  axis - 
        #convert object's pose in the camera's coordinate system/frame (transforming points from object frame to camera frame) to a 2D point in image plane and stored in imgpts
        
        corners2=corners2.astype(int)
        imgpts=imgpts.astype(int)
        # Naming a window
        cv2.namedWindow("Resized_Window", cv2.WINDOW_NORMAL)
        # Using resizeWindow()
        cv2.resizeWindow("Resized_Window", 1800, 1080)
        # Displaying the image
        img = draw(img,corners2,imgpts)
        cv2.imshow("Resized_Window", img)
        cv2.waitKey(200)
        
        
     

cv2.destroyAllWindows()

#%% print tvec_tra in x, y,z direction for each image



#%% print rotation in x, y, z direction for each image





Intrinsic Matrix (mtx):
[[603.63747782   0.         334.45612024]
 [  0.         604.91600466 243.55865732]
 [  0.           0.           1.        ]]
Translation vector (tvec) in x, y, z direction for this image:
x: -4.7952, y: -2.1344, z: 12.5302
Rotation angles (degrees) in x, y, z direction for this image:
x: 7.2310, y: -2.8955, z: -1.7569
Extrinsic Matrix (3x4) for this image:
[[ 0.99825378  0.02406034 -0.05394903 -4.79515748]
 [-0.03062043  0.99177525 -0.12427487 -2.13441456]
 [ 0.05051522  0.1257098   0.99078013 12.53023228]]
Translation vector (tvec) in x, y, z direction for this image:
x: -3.4078, y: 3.9746, z: 18.5696
Rotation angles (degrees) in x, y, z direction for this image:
x: 14.3883, y: 20.6046, z: -81.4399
Extrinsic Matrix (3x4) for this image:
[[ 0.13932472  0.97086025 -0.19498461 -3.40777466]
 [-0.92560397  0.05770328 -0.37406902  3.97456348]
 [-0.35191749  0.23259559  0.90667159 18.56959692]]
Translation vector (tvec) in x, y, z direction for this image:
x: -4.115